In [1]:
using BenchmarkTools
using StaticArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "3518907848934066751", 2, "feature")

In [4]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [5]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "18221913580574304048", 2, "feature")

## Vecchia funzione

In [19]:
function embedTraversal(cloned::Lar.Struct,obj::Lar.Struct,n::Int,suffix::String)
	for i=1:length(obj.body)
		if isa(obj.body[i],Matrix)
			mat = obj.body[i]
			d,d = size(mat)
			newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
			for h in range(1, length=d)
				for k in range(1, length=d)
					newMat[h,k]=mat[h,k]
				end
			end
			push!(cloned.body,[newMat])
		elseif (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==3
			V,FV,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,FV,EV)])
		elseif  (isa(obj.body[i],Tuple) ||isa(obj.body[i],Array))&& length(obj.body[i])==2
			V,EV = deepcopy(obj.body[i])
			dimadd = n
			ncols = size(V,2)
			nmat = zeros(dimadd,ncols)
			V = [V;zeros(dimadd,ncols)]
			push!(cloned.body,[(V,EV)])
		elseif isa(obj.body[i],Lar.Struct)
			newObj = Struct()
			newObj.box = [ [obj.body[i].box[1];zeros(dimadd)],
							[obj.body[i].box[2];zeros(dimadd)] ]
			newObj.category = (obj.body[i]).category
			push!(cloned.body,[embedTraversal(newObj,obj.body[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 2 methods)

In [21]:
@btime embedTraversal(Lar.Struct(),x,1,"New")

  8.366 μs (25 allocations: 1.81 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], Any, "15751112824915163172", Any, "feature")

In [20]:
@btime embedTraversal(Lar.Struct(),structure,1,"New")

  26.299 μs (87 allocations: 4.61 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]], [([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]]], Any, "17891135104253738189", Any, "feature")

In [ ]:
@benchmark embedTraversal(Lar.Struct(),structure,1,"New")

## Nuove funzioni

In [22]:
function embedTraversalMatrix(objBody)
	mat = objBody[i]
	d,d = size(mat)
	newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
	for h in range(1, length=d)
		for k in range(1, length=d)
			newMat[h,k]=mat[h,k]
		end
	end
	return newMat
end

embedTraversalMatrix (generic function with 2 methods)

In [23]:
function embedTraversalTupleOrArray(n,V)
	dimadd = n
	ncols = size(V,2)
	nmat = zeros(dimadd,ncols)
	V = [V;nmat]
	return V
end

embedTraversalTupleOrArray (generic function with 1 method)

In [24]:
function embedTraversalStruct(objBody)
	newObj = Struct()
	newObj.box = [ [objBody[i].box[1];zeros(dimadd)],
				   [objBody[i].box[2];zeros(dimadd)] ]
	newObj.category = (objBody[i]).category
	return newObj
end

embedTraversalStruct (generic function with 2 methods)

In [25]:
function embedTraversal(cloned,obj,n,suffix)
	objBody = obj.body
	for i=1:length(objBody)
		if isa(objBody[i],Matrix)
			newMat = embedTraversalMatrix(objBody)
			push!(cloned.body,[newMat])
		elseif (isa(objBody[i],Tuple) || isa(objBody[i],Array))
			if (length(objBody[i])==3)
				V,FV,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,FV,EV)])
			elseif (length(objBody[i])==2)
				V,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,EV)])
			end
		elseif isa(objBody[i],Struct)
			newObj = embedTraversalStruct(objBody)
			push!(cloned.body,[embedTraversal(newObj,objBody[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 2 methods)

In [26]:
@btime embedTraversal(Lar.Struct(),x,1,"New")

  8.333 μs (25 allocations: 1.81 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], Any, "2920510752912314829", Any, "feature")

In [27]:
@btime embedTraversal(Lar.Struct(),structure,1,"New")

  26.200 μs (87 allocations: 4.61 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]], [([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]]], Any, "15559375586980300763", Any, "feature")

In [28]:
@benchmark embedTraversal(Lar.Struct(),structure,1,"New")

BenchmarkTools.Trial: 
  memory estimate:  4.63 KiB
  allocs estimate:  87
  --------------
  minimum time:     26.100 μs (0.00% GC)
  median time:      28.800 μs (0.00% GC)
  mean time:        33.704 μs (1.44% GC)
  maximum time:     4.924 ms (98.63% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [29]:
@benchmark embedTraversal(Lar.Struct(),x,1,"New")

BenchmarkTools.Trial: 
  memory estimate:  1.81 KiB
  allocs estimate:  25
  --------------
  minimum time:     8.266 μs (0.00% GC)
  median time:      9.333 μs (0.00% GC)
  mean time:        9.747 μs (1.58% GC)
  maximum time:     1.579 ms (97.76% GC)
  --------------
  samples:          10000
  evals/sample:     3

In [30]:
@code_typed embedTraversal(Lar.Struct(),structure,1,"New")

CodeInfo(
1 ── %1   = Base.getfield(obj, :body)::Array
│    %2   = Base.arraylen(%1)::Int64
│    %3   = Base.sle_int(1, %2)::Bool
│    %4   = Base.ifelse(%3, %2, 0)::Int64
│    %5   = Base.slt_int(%4, 1)::Bool
└───        goto #3 if not %5
2 ──        goto #4
3 ──        goto #4
4 ┄─ %9   = φ (#2 => true, #3 => false)::Bool
│    %10  = φ (#3 => 1)::Int64
│    %11  = φ (#3 => 1)::Int64
│    %12  = Base.not_int(%9)::Bool
└───        goto #38 if not %12
5 ┄─ %14  = φ (#4 => %10, #37 => %111)::Int64
│    %15  = φ (#4 => %11, #37 => %112)::Int64
│    %16  = Base.getindex(%1, %14)::Any
│    %17  = (%16 isa Main.Matrix)::Bool
└───        goto #22 if not %17
6 ── %19  = Base.getindex(%1, %14)::Any
│    %20  = Main.embedTraversalMatrix(%19, n)::Array{Float64,2}
│    %21  = Base.getfield(cloned, :body)::Array
│    %22  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Array{Float64,2},1}, svec(Any, Int64), 0, :(:ccall), Array{Array{Float64,2},1}, 1, 1))::Array{Array{Float64,2},1}
│           B

In [30]:
@code_warntype embedTraversal(Lar.Struct(),x,1,"New")

Variables
  #self#::Core.Compiler.Const(embedTraversal, false)
  cloned::LinearAlgebraicRepresentation.Struct
  obj::LinearAlgebraicRepresentation.Struct
  n::Int64
  suffix::String
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  @_8::Any
  @_9::Any
  @_10::Any
  mat::Any
  d::Any
  newMat::Array{Float64,2}
  @_14::Union{Nothing, Tuple{Int64,Int64}}
  V::Any
  FV::Any
  EV::Any
  dimadd::Int64
  ncols::Any
  nmat::Array{Float64,2}
  newObj::Any
  h::Int64
  @_23::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64
  @_25::Bool
  @_26::Any
  @_27::Bool
  @_28::Any

Body::LinearAlgebraicRepresentation.Struct
1 ── %1   = Base.getproperty(obj, :body)::Array
│    %2   = Main.length(%1)::Int64
│    %3   = (1:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│           (@_6 = Base.iterate(%3))
│    %5   = (@_6 === nothing)::Bool
│    %6   = Base.not_int(%5)::Bool
└───        goto #30 if not %6
2 ┄─        Core.NewvarNode(:(@_8))
│           Core.N

## Modifiche

"""\
embedTraversalMatrix(objBody::Matrix,n::Int)::Matrix\
Funzione chiamata se il corpo della struttura che si sta modellando è una matrice. Trasforma la matrice aumentandone la dimensione del valore n.\
"""

In [72]:
@inline function embedTraversalMatrix(objBody,n)
	mat = objBody
	d,d = size(mat)
	newMat = Matrix{Float64}(LinearAlgebra.I, d+n, d+n)
	@inbounds for h in range(1, length=d)
		@inbounds for k in range(1, length=d)
			newMat[h,k]=mat[h,k]
		end
	end
	return newMat
end

embedTraversalMatrix (generic function with 1 method)

"""\
embedTraversalTupleOrArray(objBody::Union{Tuple,Array},V)\
Funzione chiamata se il corpo della struttura che si sta modellando è una tupla o un array. Genera e restituisce il parametro V.\
"""

In [67]:
@inline function embedTraversalTupleOrArray(n,V)
	dimadd = n
	ncols = size(V,2)
	nmat = zeros(dimadd,ncols)
	V = [V;nmat]
	return V
end

embedTraversalTupleOrArray (generic function with 1 method)

"""\
embedTraversalStruct(objBody::Struct)::Struct\
Funzione chiamata se il corpo della struttura che si sta modellando è una struttura. Genera(a partire dalla struttura in ingresso objBody) e restituisce la struttura newObj.\
"""

In [68]:
@inline function embedTraversalStruct(objBody)
	newObj = Struct()
	newObj.box = [ [objBody.box[1];zeros(dimadd)],
				   [objBody.box[2];zeros(dimadd)] ]
	newObj.category = objBody.category
	return newObj
end

embedTraversalStruct (generic function with 1 method)

In [85]:
@inline function embedTraversal(cloned::Lar.Struct,obj::Lar.Struct,n::Int,suffix::String)
	objBody = obj.body
	@inbounds @async for i=1:length(objBody)
		if isa(objBody[i],MMatrix)
			newMat = embedTraversalMatrix(objBody[i],n)
			push!(cloned.body,[newMat])
		elseif (isa(objBody[i],Tuple) || isa(objBody[i],Array))
			if (length(objBody[i])==3)
				V,FV,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,FV,EV)])
			elseif (length(objBody[i])==2)
				V,EV = deepcopy(objBody[i])
				V = embedTraversalTupleOrArray(n,V)
				push!(cloned.body,[(V,EV)])
			end
		elseif isa(objBody[i],Struct)
			newObj = embedTraversalStruct(objBody[i])
			push!(cloned.body,[embedTraversal(newObj,objBody[i],obj.dim+n,suffix)])
		end
	end
	return cloned
end

embedTraversal (generic function with 1 method)

In [86]:
@btime embedTraversal(Lar.Struct(),x,1,"New")

  5.300 μs (9 allocations: 1.08 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], Any, "13823364604560428653", Any, "feature")

In [87]:
@btime embedTraversal(Lar.Struct(),structure,1,"New")

  5.300 μs (9 allocations: 1.08 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]], [([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])], [[6.123233995736766e-17 -1.0 0.0 0.0; 1.0 6.123233995736766e-17 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]]], Any, "17762258424708736319", Any, "feature")

In [ ]:
@benchmark embedTraversal(Lar.Struct(),structure,1,"New")